In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
import os
import seaborn as sns

In [ ]:
print(os.listdir('/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset'))

In [ ]:
SIZE = 128

In [ ]:
train_images = []
train_labels = [] 
for directory_path in glob.glob("/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/train/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
#         print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
# test
test_images = []
test_labels = [] 
for directory_path in glob.glob("/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/train/*"):
    fruit_label = directory_path.split("/")[-1]
    print(fruit_label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
        
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
# Save the preprocessed images as a NumPy array
np.save("/kaggle/working/train_images.npy", train_images)

# Save the labels as a CSV file
df = pd.DataFrame({"label": train_labels})
df.to_csv("/kaggle/working/train_labels.csv", index=False)


# Save the preprocessed images as a NumPy array
np.save("/kaggle/working/test_images.npy", test_images)

# Save the labels as a CSV file
df = pd.DataFrame({"label": train_labels})
df.to_csv("/kaggle/working/test_labels.csv", index=False)

In [ ]:
# Load the processed images and labels from the Kaggle dataset
train_images = np.load("/kaggle/working/train_images.npy")
train_labels = pd.read_csv("/kaggle/working/train_labels.csv")["label"].values

test_images = np.load("/kaggle/working/test_images.npy")
test_labels = pd.read_csv("/kaggle/working/test_labels.csv")["label"].values


In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

class_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

# Print the mapping
print(class_mapping)

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

###################################################################
# Normalize pixel values to between 0 and 1
# x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)


In [ ]:
activation = 'relu'

feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation = activation,input_shape = (SIZE, SIZE, 3)))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(32, 3, activation = activation))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D(pool_size=3))

feature_extractor.add(Conv2D(64, 3, activation = activation))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(64, 3, activation = activation))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D(pool_size=3))

feature_extractor.add(Flatten())

#Add layers for deep learning prediction
x = feature_extractor.output  
x = Dense(512, activation = activation)(x)
prediction_layer = Dense(6, activation = 'softmax')(x)


In [ ]:
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer='adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary()) 

In [ ]:
#Now, let us use features from convolutional network for RF
X_for_RF = feature_extractor.predict(x_train) #This is out X input to RF

#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

# Train the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding


In [ ]:
#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)
#Now predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_feature)
#Inverse le transform to get original label back. 
prediction_RF = le.inverse_transform(prediction_RF)


In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))

In [ ]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, prediction_RF)
#print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
#Check results on a few select images
#n=5 #dog park. RF works better than CNN
# for n in range(10000,10900,50):
n=1000#Select the index of image to be loaded for testing
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_features=feature_extractor.predict(input_img)
prediction_RF = RF_model.predict(input_img_features)[0] 
prediction_RF = le.inverse_transform([prediction_RF])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction_RF)
print("The actual label for this image is: ", test_labels[n])

In [ ]:
# save the model to disk
import pickle
filename = 'finalized_RF2_model.pkl'
pickle.dump(RF_model, open(filename, 'wb'))